In [123]:
import pandas as pd
import json
import numpy as np
import os
import glob
import re
import urllib
from bs4 import BeautifulSoup
from pathlib import Path


In [2]:
def ig_unknown(file, hl_min, hl_max):
    return "Feature not implemented yet. Write an issue to GitHub."

In [ ]:
def ig_isotope(measured_df, mat_list, hl_min, hl_max, name):
    precursores = mat_list.split(',')
    isotopes = []
    with open("database/irr_products.json") as f:
        irradiation_products = json.load(f)
    
    for precursor in precursores:
        try:
            isotopes += irradiation_products[precursor]
        
        except:
            irradiation_products[precursor] = load_irr_prod(precursor)
            with open("database/irr_products.json", 'w') as json_out:
                json.dump(irradiation_products, json_out)

    # get unique only
    isotopes = list(set(isotopes))

    db_parameters = ["half-life", "sigm_half-life", "lambda", "sigm_lambda", "isotope", "Ig", "sigm_Ig", "E_tab", "sigm_E_tab"]

    measured_df[[db_parameters]] = np.nan*np.ones(shape=(measured_df.shape[0], len(db_parameters)))

    for isotope in isotopes:
        measured_df = add_db_info(measured_df, isotope)
    
    measured_df.to_csv(f"out/{name}wDBinfo.csv")

In [ ]:
def ig_nat_element(file, mat_list, hl_min, hl_max):
    return "Feature not implemented yet. Write an issue to GitHub."

In [ ]:
def ig_element_spec(file, mat_list, hl_min, hl_max):
    return "Feature not implemented yet. Write an issue to GitHub."

In [ ]:
def ig_material(file, mat_list, hl_min, hl_max):
    return "Feature not implemented yet. Write an issue to GitHub."

In [76]:
#def get_Igamma(file, material, hl_min, hl_max):
#    df = pd.read_csv(file,header=0,index_col=0)
#    name = file.split('.')[0]
#    mat_type, mat_list = material.split(',', maxindex=1)
#    switcher = {
#        0: ig_unknown(df, hl_min, hl_max, name),
#        "isotope": ig_isotope(df, mat_list, hl_min, hl_max, name), 
#        "element_natural": ig_nat_element(df, mat_list, hl_min, hl_max, name),
#        "element_spec": ig_spec_element(df, mat_list, hl_min, hl_max, name),
#        "material": ig_material(df, mat_list, hl_min, hl_max, name)
#    }
#    switcher.get(mat_type, "Invalid material given")

In [216]:
def append_Igamma_dir(parsed_dir):
    if not os.path.isdir(parsed_dir):
        raise Exception(f"Directory '{parsed_dir}' not found. Consider creating it and moving parsed files there.")

    Path("./reports_Ig").mkdir(parents=True, exist_ok=True)
    for parsed_file in glob.iglob(f"{parsed_dir}/*.csv"):
        append_Igamma(parsed_file)

In [217]:
def append_Igamma(file):
    print(file)
    A, element = re.split(r'(\d+)(\w+)', file)[-3:-1]
    print(A)
    print(element)

    Path("./ig_db").mkdir(parents=True, exist_ok=True)
    if os.path.isfile(f"ig_db/{A}{element}.csv"):
        #ig_df = pd.read_csv(f"ig_db/{ig_file_name}", header=None, index_col=None)
        print("Reading the csv with gammas.")
    else:
        Path("./downloads").mkdir(parents=True, exist_ok=True)
        if os.path.isfile(f"downloads/{A}{element}.html"):
            extract_Igamma(A, element)
            #ig_df = pd.read_csv(f"ig_db/{ig_file_name}", header=None, index_col=None)
            print("Reading the csv with gammas.")
        else:
            download(A, element)
            extract_Igamma(A, element)
            #ig_df = pd.read_csv(f"ig_db/{ig_file_name}", header=None, index_col=None)
            print("Reading the csv with gammas.")

In [218]:
def getZ(element):
    element_Z ={
        "U": 92,
        "Pu": 94
    }
    return element_Z[element]

In [219]:
def download(A, element):
    #Path(f'downloads/{A}{element}.html').touch()
    url = f"http://nucleardata.nuclear.lu.se/toi/nuclide.asp?iZA={getZ(element)}0{A}"
    urllib.request.urlretrieve (url, f"downloads/{A}{element}.html")
    print(f"File {A}{element}.html downloaded.")

In [248]:
def extract_Igamma(A, element):
    html_file = open(f"downloads/{A}{element}.html", "r")
    soup = BeautifulSoup(html_file.read(), 'lxml')
    
    gammas_table = soup.find_all("table")[4]
    gammas_rows = gammas_table.find_all('tr')[3:-1]
    energy = []
    sigm_energy = []
    i = []
    sigm_i = []

    for row in gammas_rows:
        cells = row.find_all('td')
        
        e_val = cells[0].get_text(strip=True)
        i_val = cells[1].get_text(strip=True)
        if i_val == '':
            ig_val = float('NaN')
            sigm_ig_val = float('NaN')
        else:
            ig_val = float(i_val[:-1])
            sigm_ig_val = float(i_val[-1])
        
        print("Energy is")
        print(e_val[:-1])
        print("dE")
        print(e_val[-1])
        print("Ig")
        print(ig_val)
        print("sigm_Ig")
        print(sigm_ig_val)
        #try:
        #    acc = i_val[-1]
        #except:
        #    acc = float("NaN")
        #print(i_val[-1])

        energy.append(float(e_val[:-1]))
        sigm_energy.append(int(e_val[-1]))
        i.append(ig_val)
        
        sigm_i.append(sigm_ig_val)


    df_dict = {
        "E": energy,
        "sigm_E": sigm_energy, 
        "Ig": i,
        "sigm_Ig": sigm_i
        }
    df = pd.DataFrame(df_dict)
    df_name = f'ig_db/{A}{element}.csv'
    df.to_csv(df_name)
    #file = open(f"downloads/{A}{element}.html", 'r')
    #copy_line = False
    #gamma_lines = []
    #for line in file.readlines():
    #    if "A NAME=\"g\"" in line:
    #        copy_line = True
    #        continue
    #    if copy_line:
    #        gamma_lines.append(line)
    #    
    #    if "A NAME=\"x\"" in line:
    #        break
    #    
    #file.close()

    #with open(f"ig_db/{A}{element}.csv", "w") as f:
    #    for ig in gamma_lines:
    #        f.write(f"{ig}\n")
    #    f.close()
    print(f"Ig extracted from file 'downloads/{A}{element}.html' into '{df_name}'.")

In [259]:
html_doc = open("downloads/238U.html", "r")

soup = BeautifulSoup(html_doc.read(), 'lxml')
gammas_table = soup.find_all("table")[4]
gammas_rows = gammas_table.find_all('tr')[3:-1]
gamma_energy_list = []
gamma_i_list = []

for row in gammas_rows:
    cells = row.find_all('td')
    e_val = cells[0].get_text()
    print(re.split(r'\\', e_val))

    gamma_energy_list.append(e_val)
    gamma_i_list.append(cells[1].get_text(strip=False))


print(gamma_energy_list)
print(gamma_i_list)

['49.55\xa06\xa0']
['113.5\xa01\xa0']
['49.55\xa06\xa0', '113.5\xa01\xa0']
['0.064\xa08\xa0', '0.0102\xa015\xa0']


In [250]:
append_Igamma_dir("parsed_reports")

parsed_reports/238U.csv
238
U
Reading the csv with gammas.
parsed_reports/239Pu.csv
239
Pu
Energy is
0.0768
dE
5
Ig
nan
sigm_Ig
nan
Energy is
12.963
dE
2
Ig
nan
sigm_Ig
nan
Energy is
30.037
dE
3
Ig
0.000217
sigm_Ig
6.0
Energy is
38.661
dE
2
Ig
0.0105
sigm_Ig
2.0
Energy is
40.41
dE
5
Ig
0.0001621
sigm_Ig
6.0
Energy is
42.088
dE
4
Ig
0.000165
sigm_Ig
5.0
Energy is
46.204
dE
6
Ig
0.00074
sigm_Ig
7.0
Energy is
46.6252
dE
0
Ig
5.8e-05
sigm_Ig
4.0
Energy is
47.560
dE
3
Ig
0.0
sigm_Ig
6.0
Energy is
51.624
dE
1
Ig
0.0271
sigm_Ig
5.0
Energy is
54.030
dE
4
Ig
0.000197
sigm_Ig
3.0
Energy is
56.828
dE
3
Ig
0.0011302
sigm_Ig
5.0
Energy is
65.7231
dE
9
Ig
4.561e-05
sigm_Ig
4.0
Energy is
67.6731
dE
0
Ig
0.000164
sigm_Ig
3.0
Energy is
68.699
dE
3
Ig
0.0003
sigm_Ig
1.0
Energy is
68.73
dE
2
Ig
0.00011
sigm_Ig
6.0
Energy is
74.961
dE
0
Ig
3.8e-05
sigm_Ig
6.0
Energy is
77.598
dE
2
Ig
0.00041
sigm_Ig
2.0
Energy is
78.4221
dE
1
Ig
0.000141
sigm_Ig
6.0
Energy is
89.39
dE
6
Ig
0.0
sigm_Ig
2.0
Energy is
89.648

ValueError: could not convert string to float: '<0.00005'